# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [ ]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from newsapi import NewsApiClient
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import regex as re

import nltk
nltk.download('stopwords')
nltk.download('punkt')

%matplotlib inline

In [ ]:
# Read your api key environment variable
load_dotenv('')
api_key = os.getenv('NEWS_API_KEY')

In [ ]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [ ]:
# Fetch the Bitcoin news articles
btc_articles = newsapi.get_everything(q='bitcoin')

In [ ]:
# Fetch the Ethereum news articles
eth_articles = newsapi.get_everything(q='ethereum')

In [ ]:
# Create the Bitcoin sentiment scores DataFrame
btc_sentiments = []

for article in btc_articles['articles']:
    try:
        text = article["content"]
        date = article["publishedAt"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pas = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pas,
            "negative": neg,
            "neutral": neu
            
        })
    except AttributeError:
        pass
    
    btc_df = pd.DataFrame(btc_sentiments)
    
    cols = ["date", "text", "compound", "positive", "negative", "neutral"]
    btc_df = btc_df[cols]

In [ ]:
btc_df.head()

In [ ]:
# Create the Ethereum sentiment scores DataFrame
eth_sentiments = []

for article in eth_articles['articles']:
    try:
        text = article["content"]
        date = article["publishedAt"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pas = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pas,
            "negative": neg,
            "neutral": neu
            
        })
    except AttributeError:
        pass
    
    eth_df = pd.DataFrame(eth_sentiments)
    
    cols = ["date", "text", "compound", "positive", "negative", "neutral"]
    eth_df = eth_df[cols]

In [ ]:
 eth_df.head()

In [ ]:
# Describe the Bitcoin Sentiment
btc_df.describe()

In [ ]:
# Describe the Ethereum Sentiment
eth_df.describe()

### Questions:

Q: Which coin had the highest mean positive score?

A: btc 0.182980

Q: Which coin had the highest compound score?

A: eth 0.735100

Q. Which coin had the highest positive score?

A: btc 0.735100

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [ ]:
# Complete the tokenizer function
def tokenizer(article):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', article)
   
    # Create a tokenized list of the words
    words = word_tokenize(re_clean)
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    tokens = [word.lower() for word in lem if word.lower() not in sw]
    
    # Remove the stop words
    
    
    return tokens

In [ ]:
# Create a new tokens column for Bitcoin
btc_df["token"] = btc_df.text.apply(tokenizer)


In [ ]:
btc_df.head()

In [ ]:
# Create a new tokens column for Ethereum
eth_df["token"] = eth_df.text.apply(tokenizer)
eth_df.head()

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
N=2
btc_grams = ngrams(tokenizer(btc_df.text.str.cat()), n=2)
Counter(btc_grams).most_common(10)

In [ ]:
# Generate the Ethereum N-grams where N=2
N=2
eth_grams = ngrams(tokenizer(eth_df.text.str.cat()), n=2)
Counter(eth_grams).most_common(10)

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
btc_10 = tokenizer(btc_df.text.str.cat())
token_count(btc_10, N=10)

In [ ]:
# Use token_count to get the top 10 words for Ethereum
eth_10 = tokenizer(eth_df.text.str.cat())
token_count(eth_10, N=10)

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud

def word_cloud(text, title):
    wordcloud = WordCloud(colormap="RdYlBu").generate(text)
    plt.imshow(wordcloud)
    plt.axis("off")
    fontdict = {"fontsize": 20, "fontweight": "bold"}
    plt.title(title, fontdict=fontdict)
    plt.show()

btc_wc = word_cloud(btc_df.text.str.cat(), "BTC Word Cloud")

In [ ]:
# Generate the Ethereum word cloud
eth_wc = word_cloud(eth_df.text.str.cat(), "ETH Word Cloud")

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
btc_text_ner = btc_df.text.str.cat()

In [ ]:
# Run the NER processor on all of the text
btc_doc = nlp(btc_text_ner)

# Add a title to the document
btc_doc.user_data['title'] = "Bitcoin Ner"

In [ ]:
# Render the visualization
displacy.render(btc_doc, style='ent')

In [ ]:
# List all Entities

for ent in btc_doc.ents:
    print(ent.text,ent.label_)

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
eth_text_ner = eth_df.text.str.cat()

In [ ]:
# Run the NER processor on all of the text
eth_doc = nlp(eth_text_ner)

# Add a title to the document
eth_doc.user_data['title'] = "Ethereum Ner"

In [ ]:
# Render the visualization
displacy.render(eth_doc, style='ent')

In [ ]:
# List all Entities
for ent in eth_doc.ents:
    print(ent.text,ent.label_)

---